# セットアップ

In [1]:
!sudo apt-get update
!sudo apt-get install -y xvfb ffmpeg freeglut3-dev
!pip install 'imageio==2.4.0'
!pip install pyvirtualdisplay
!pip install tf-agents[reverb]
!pip install pyglet

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Ign:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [2,944 kB]
Get:14 https://r2u.st

In [19]:
from __future__ import absolute_import, division, print_function

import base64
import imageio
import IPython
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
import pyvirtualdisplay
import reverb

import tensorflow as tf

from tf_agents.agents.dqn import dqn_agent
from tf_agents.drivers import py_driver
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.networks import sequential
from tf_agents.policies import py_tf_eager_policy
from tf_agents.policies import random_tf_policy
from tf_agents.replay_buffers import reverb_replay_buffer
from tf_agents.replay_buffers import reverb_utils
from tf_agents.trajectories import trajectory
from tf_agents.trajectories import time_step as ts
from tf_agents.specs import tensor_spec
from tf_agents.utils import common

In [3]:
import sys
from os import mkdir
from os.path import exists
from sys import exc_info
import numpy as np
from enum import Enum, IntEnum, auto
from collections import Counter
from functools import reduce
# !pip install gym
import gym
from gym import spaces
from gym.utils import seeding

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import io

# import keras
# !pip install wandb

# !pip install dopamine-rl
# !pip install keras-rl2
# !pip install keras-rl
# !pip install keras
# !git clone https://github.com/matthiasplappert/keras-rl.git
# !pip install ./keras-rl
# !pip install rl
# import importlib
# import kerasrl/rl
# from kerasrl.rl.agents.dqn import DQNAgent
# from kerasrl.rl.policy import BoltzmannQPolicy
# from kerasrl.rl.memory import SequentialMemory
# from kerasrl.rl.callbacks import TrainIntervalLogger, TrainEpisodeLogger
# from kerasrl.rl.callbacks import TrainIntervalLogger, TrainEpisodeLogger

In [232]:
num_iterations = 20000 #反復回数 @param {type:"integer"}
# num_iterations = 200 #反復回数 @param {type:"integer"}

initial_collect_steps = 1000  #初期収集ステップ数 @param {type:"integer"}
collect_steps_per_iteration = 1  #反復あたりの収集ステップ数 @param {type:"integer"}
replay_buffer_max_length = 100000  #再生バッファの最大長 @param {type:"integer"}

batch_size = 64  #バッチサイズ @param {type:"integer"}
learning_rate = 1e-3  #学習率 @param {type:"number"}
log_interval = 200  #ログの間隔 @param {type:"integer"}

num_eval_episodes = 10  #評価エピソード数 @param {type:"integer"}
eval_interval = 1000  #評価間隔 @param {type:"integer"}

# 中間

In [6]:
class Hand(IntEnum):
    R = 0
    S = 1
    P = 2

class Strategy(Enum):
    RANDOM = auto()
    HUMAN = auto()
    PUMPKIN = auto()
    MAXIMUM = auto()
    NASH = auto()
    MEMORY = auto()
    MAJORITY = auto()

In [7]:
class RandomStrategy:
    hands = [Hand.R, Hand.S, Hand.P]

    def __init__(self, np_random):
        self.np_random = np_random

    def get_hand(self, user_hand):
        return self.np_random.choice(RandomStrategy.hands)

    def reset(self):
        pass

class HumanStrategy:
    # 芳沢光雄(2009)『ジャンケンに関する研究結果』による
    ratio = [0.350, 0.317, 0.333]
    hands = [Hand.R, Hand.S, Hand.P]

    def __init__(self, np_random):
        self.np_random = np_random

    def get_hand(self, user_hand):
        return self.np_random.choice(HumanStrategy.hands, p=HumanStrategy.ratio)

    def reset(self):
        pass

class NashStrategy:
    ratio = [0.4, 0.4, 0.2]
    hands = [Hand.R, Hand.S, Hand.P]

    def __init__(self, np_random):
        self.np_random = np_random

    def get_hand(self, user_hand):
        return self.np_random.choice(NashStrategy.hands, p=NashStrategy.ratio)

    def reset(self):
        pass

In [233]:
class RSP125(gym.Env):
    metadata = {
        'render.modes': ['human', 'ansi']
    }

    action_space = spaces.Discrete(3) # グー、チョキ、パー
    reward_range = [0, 5] # 報酬の最小値、最大値
    # observation_space = spaces.Box(low=0, high=100, shape=(4,), dtype='float32')
    observation_space = spaces.Box(low=0, high=2, shape=([1,10]), dtype='int64')

    reward_space = spaces.Box(low=0, high=500, shape=(1,), dtype=np.float32)
    # 'step_type': TensorShape([]),
    # 'reward': TensorShape([]),
    # 'discount':

    # def time_step_spec(self):
    #     return self._time_step_spec

    def __init__(self, strategy=Strategy.HUMAN, goal=100):
        super().__init__()

        self.seed()

        self.strategy = None
        if strategy == Strategy.RANDOM:
            self.strategy = RandomStrategy(self.np_random)
        elif strategy == Strategy.HUMAN:
            self.strategy = HumanStrategy(self.np_random)
        elif strategy == Strategy.PUMPKIN:
            self.strategy = PumpkinStrategy(self.np_random)
        elif strategy == Strategy.MAXIMUM:
            self.strategy = MaximumStrategy(self.np_random)
        elif strategy == Strategy.NASH:
            self.strategy = NashStrategy(self.np_random)
        elif strategy == Strategy.MAJORITY:
            self.strategy = MajorityStrategy(self.np_random)
        elif strategy == Strategy.MEMORY:
            self.strategy = MemoryStrategy(self.np_random)

        self.goal = goal
        self.user_hands = []
        self.enemy_hands = []
        self.reset()

        # # 勝手に作成中
        self.observation_space = spaces.Box(low=0, high=2, shape=(10,), dtype=np.int64)
        self.action_space = spaces.Discrete(3)
        self._time_step_spec = ts.TimeStep(
            step_type=tensor_spec.TensorSpec(shape=(1,), dtype=tf.int32, name='step_type'),
            reward=tensor_spec.TensorSpec(shape=(1,), dtype=tf.float32, name='reward'),
            discount=tensor_spec.BoundedTensorSpec(shape=(1,), dtype=tf.float32, minimum=0.0, maximum=1.0, name='discount'),
            observation=tensor_spec.BoundedTensorSpec(shape=(1, 10), dtype=tf.int64, minimum=0, maximum=2, name='observation')
        )
        # self._time_step_spec = ts.TimeStep(
        #     step_type=tensor_spec.BoundedTensorSpec(shape=[1], dtype=tf.int32, minimum=0, maximum=2),
        #     reward=tensor_spec.TensorSpec(shape=[1], dtype=tf.float32),
        #     discount=tensor_spec.BoundedTensorSpec(shape=[1], dtype=tf.float32, minimum=0.0, maximum=1.0),
        #     observation=tensor_spec.TensorSpec(shape=[10], dtype=tf.float32)
        # )

    def reset(self):
        self.user_position = 0
        self.enemy_position  = 0
        self.user_hand = Hand.R   # 最初はグー
        self.user_positions = [self.user_position]
        self.enemy_hand = Hand.R  # 最初はグー
        self.enemy_positions = [self.enemy_position]
        self.strategy.reset()
        self.done = False
        # observation = [self.user_position, self.enemy_position, self.user_hand, self.enemy_hand]
        self.observation = np.array([2, 0, 1, 2, 1, 0, 2, 1, 0, 1], dtype=np.int64)
        # self.observation = ts.restart([2, 0, 1, 2, 1, 0, 2, 1, 0, 1])
        return self.observation
        # return self.observation, 0, self.done, {}

    def seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed) # シードから乱数ジェネレーターを生成し、ジェネレーターとシードを返す
        return [seed]

    def update_observation(self, userhand, enemyhand, obs):
      obs = obs[2:]
      obs = np.append(obs,userhand)
      obs = np.append(obs, enemyhand)
      return obs

    def step(self, action):
        # self.enemy_hand = self.strategy.get_hand(self.user_hand)
        self.enemy_hand = Hand(self.strategy.get_hand(self.user_hand))
        self.enemy_hands += [self.enemy_hand]
        self.user_hand = Hand(action)
        self.user_hands += [self.user_hand]

        # あいこの場合
        if self.user_hand == self.enemy_hand:
            # observation = [self.user_position, self.enemy_position, self.user_hand, self.enemy_hand]
            self.observation = self.update_observation(action, self.strategy.get_hand(self.user_hand), self.observation)
            reward = 0
            self.done = False

        # プレーヤーが勝つ場合
        elif self.user_hand == Hand.R and self.enemy_hand == Hand.S:
            self.user_position += 1
            # observation = [self.user_position, self.enemy_position, self.user_hand, self.enemy_hand]
            self.observation = self.update_observation(action, self.strategy.get_hand(self.user_hand), self.observation)
            if self.user_position >= 100:
                reward = 1000
                self.done = True
            else:
                reward = 0
                self.done = False

        elif self.user_hand == Hand.S and self.enemy_hand == Hand.P:
            self.user_position += 2
            # observation = [self.user_position, self.enemy_position, self.user_hand, self.enemy_hand]
            self.observation = self.update_observation(action, self.strategy.get_hand(self.user_hand), self.observation)
            if self.user_position >= 100:
                reward = 1000
                self.done = True
            else:
                reward = 0
                self.done = False

        elif self.user_hand == Hand.P and self.enemy_hand == Hand.R:
            self.user_position += 5
            # observation = [self.user_position, self.enemy_position, self.user_hand, self.enemy_hand]
            self.observation = self.update_observation(action, self.strategy.get_hand(self.user_hand), self.observation)
            if self.user_position >= 100:
                reward = 1000
                self.done = True
            else:
                reward = 0
                self.done = False

        # 敵が勝つ場合
        elif self.user_hand == Hand.R and self.enemy_hand == Hand.P:
            self.enemy_position += 5
            # observation = [self.user_position, self.enemy_position, self.user_hand, self.enemy_hand]
            self.observation = self.update_observation(action, self.strategy.get_hand(self.user_hand), self.observation)
            if self.enemy_position >= 100:
                reward = 0
                self.done = True
            else:
                reward = 0
                self.done = False

        elif self.user_hand == Hand.S and self.enemy_hand == Hand.R:
            self.enemy_position += 1
            # observation = [self.user_position, self.enemy_position, self.user_hand, self.enemy_hand]
            self.observation = self.update_observation(action, self.strategy.get_hand(self.user_hand), self.observation)
            if self.enemy_position >= 100:
                reward = 0
                self.done = True
            else:
                reward = 0
                self.done = False

        elif self.user_hand == Hand.P and self.enemy_hand == Hand.S:
            self.enemy_position += 2
            # observation = [self.user_position, self.enemy_position, self.user_hand, self.enemy_hand]
            self.observation = self.update_observation(action, self.strategy.get_hand(self.user_hand), self.observation)
            if self.enemy_position >= 100:
                reward = 0
                self.done = True
            else:
                reward = 0
                self.done = False

        self.user_positions += [self.user_position]
        self.enemy_positions += [self.enemy_position]

        return self.observation, reward, self.done, {}
        # obs, reward, done, info = step(action)   infoは好きに使える辞書型

    def e2s(self, e):
        if e == Hand.R:
            s = 'グー'
        elif e == Hand.S:
            s = 'チョキ'
        elif e == Hand.P:
            s = 'パー'
        return s

    def render(self, mode='human', close=False):
        if mode == 'ansi':
            outfile = io.StringIO()
        elif mode == 'human':
            outfile = sys.stdout
        else:
            # just raise an exception
            super().render(mode=mode)

        strs = [i for i in range(5)]
        strs[0] = '\n'
        strs[1] = 'プレーヤーの手：' + self.e2s(self.user_hand) + '、'
        strs[2] = '敵の手：' + self.e2s(self.enemy_hand) + '\n'
        strs[3] = 'プレーヤーの位置：' + str(self.user_position) + '、'
        strs[4] = '敵の位置：' + str(self.enemy_position) + '\n'
        outfile.write(''.join(strs))
        return outfile

    def close(self):
        # just return
        super().close()

    def play(self):
        self.reset()
        while not self.done:
            print('')
            print('じゃーんけーん')
            self.step(self.__input())
            self.render()
        print('また遊ぼうね！')

    def __input(self):
        while True:
            print('[グー：0、チョキ：1、パー：2]')
            i = input()
            if i in ['0', '1', '2']:
                break
        return int(i)


    # 勝手に追加
    def observation_spec(self):
        return self._time_step_spec.observation
    def action_spec(self):
        return tensor_spec.BoundedTensorSpec(shape=(3,), dtype=tf.int64, minimum=0, maximum=2)
    def _reset(self):
        observation = np.array( np.random.randint(0, 3, size=(1, 10)) )
        return ts.restart(observation)
    def _step(self, action):
        observation = np.random.randint(0, 3, size=(1, 10))
        reward = np.random.random(size=(1,))
        step_type = np.array([ts.StepType.MID], dtype=np.int)
        discount = np.array([1.0], dtype=np.float32)
        return ts.TimeStep(
            step_type=step_type,
            reward=reward,
            discount=discount,
            observation=observation
        )
    def time_step_spec(self):
        return self._time_step_spec

In [234]:
env = suite_gym.wrap_env(RSP125())
print('Observation Spec:')
print(env.observation_space)
print('Reward Spec:')
print(env.reward_range)
print('Action Spec:')
print(env.action_space)
env.observation_space.sample()

print()
print()

# print(env.time_step_spec_custom())
print(env.time_step_spec())
print("-- env.reset() --")
print(env.reset())

print()
print()

time_step_test = env.reset()
print('Time step:')
print(time_step_test)
# env.reset()

action = np.array(1, dtype=np.int64)
next_time_step_test = env.step(action)
print('Next time step:')
print(next_time_step_test)
print("[MyPosi,エネposi,MyHan,エネhan]")

print()

action = np.array(2, dtype=np.int64)
next_next_time_step_test = env.step(action)
print('Next time step:')
print(next_next_time_step_test)
print("[MyPosi,エネposi,MyHan,エネhan]")

Observation Spec:
Box(0, 2, (10,), int64)
Reward Spec:
[0, 5]
Action Spec:
Discrete(3)


TimeStep(
{'step_type': ArraySpec(shape=(), dtype=dtype('int32'), name='step_type'),
 'reward': ArraySpec(shape=(), dtype=dtype('float32'), name='reward'),
 'discount': BoundedArraySpec(shape=(), dtype=dtype('float32'), name='discount', minimum=0.0, maximum=1.0),
 'observation': BoundedArraySpec(shape=(10,), dtype=dtype('int64'), name='observation', minimum=0, maximum=2)})
-- env.reset() --
TimeStep(
{'step_type': array(0, dtype=int32),
 'reward': array(0., dtype=float32),
 'discount': array(1., dtype=float32),
 'observation': array([2, 0, 1, 2, 1, 0, 2, 1, 0, 1])})


Time step:
TimeStep(
{'step_type': array(0, dtype=int32),
 'reward': array(0., dtype=float32),
 'discount': array(1., dtype=float32),
 'observation': array([2, 0, 1, 2, 1, 0, 2, 1, 0, 1])})
Next time step:
TimeStep(
{'step_type': array(1, dtype=int32),
 'reward': array(0., dtype=float32),
 'discount': array(1., dtype=float32),
 'obser

In [235]:
# 環境のリセットとステップを実行
initial_observation = env.reset()
print("Initial Observation:", initial_observation)

action = env.action_space.sample()
print("Sample Action:", action)

observation, reward, done, info = env.step(action)
print("Next Observation:", observation)
print("Reward:", reward)
print("Done:", done)
print("Info:", info)



Initial Observation: TimeStep(
{'step_type': array(0, dtype=int32),
 'reward': array(0., dtype=float32),
 'discount': array(1., dtype=float32),
 'observation': array([2, 0, 1, 2, 1, 0, 2, 1, 0, 1])})
Sample Action: 1
Next Observation: 1
Reward: 0.0
Done: 1.0
Info: [1 2 1 0 2 1 0 1 1 1]


In [236]:
train_py_env = RSP125()
eval_py_env = RSP125()

wrap_train_py_env = suite_gym.wrap_env(train_py_env)
wrap_eval_py_env = suite_gym.wrap_env(eval_py_env)

train_env = tf_py_environment.TFPyEnvironment(wrap_train_py_env)
eval_env = tf_py_environment.TFPyEnvironment(wrap_eval_py_env)

# エージェント

In [238]:
fc_layer_params = (100, 50)
# action_tensor_spec = tensor_spec.from_spec(env.action_space)
# num_actions = action_tensor_spec.maximum - action_tensor_spec.minimum + 1
num_actions = 3

# Define a helper function to create Dense layers configured with the right
# activation and kernel initializer.
def dense_layer(num_units):
  return tf.keras.layers.Dense(
      num_units,
      activation=tf.keras.activations.relu,
      kernel_initializer=tf.keras.initializers.VarianceScaling(
          scale=2.0, mode='fan_in', distribution='truncated_normal'))

# QNetwork consists of a sequence of Dense layers followed by a dense layer
# with `num_actions` units to generate one q_value per available action as
# its output.
dense_layers = [dense_layer(num_units) for num_units in fc_layer_params]
q_values_layer = tf.keras.layers.Dense(
    num_actions,
    activation=None,
    kernel_initializer=tf.keras.initializers.RandomUniform(
        minval=-0.03, maxval=0.03),
    bias_initializer=tf.keras.initializers.Constant(-0.2))
# q_net = sequential.Sequential(dense_layers + [q_values_layer])
# q_net = sequential.Sequential(dense_layers + [q_values_layer, tf.keras.layers.Lambda(lambda x: tf.squeeze(x, axis=-2))])
q_net = sequential.Sequential(dense_layers + [q_values_layer])


In [239]:
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

train_step_counter = tf.Variable(0)

print(train_env.time_step_spec())
print(train_env.action_spec())

agent = dqn_agent.DqnAgent(
    train_env.time_step_spec(),
    train_env.action_spec(),
    q_network=q_net,
    optimizer=optimizer,
    td_errors_loss_fn=common.element_wise_squared_loss,
    train_step_counter=train_step_counter
  )

agent.initialize()

TimeStep(
{'step_type': TensorSpec(shape=(), dtype=tf.int32, name='step_type'),
 'reward': TensorSpec(shape=(), dtype=tf.float32, name='reward'),
 'discount': BoundedTensorSpec(shape=(), dtype=tf.float32, name='discount', minimum=array(0., dtype=float32), maximum=array(1., dtype=float32)),
 'observation': BoundedTensorSpec(shape=(10,), dtype=tf.int64, name='observation', minimum=array(0), maximum=array(2))})
BoundedTensorSpec(shape=(), dtype=tf.int64, name='action', minimum=array(0), maximum=array(2))


# ポリシー

In [240]:
eval_policy = agent.policy # 評価とデプロイに使用される主なポリシー。
collect_policy = agent.collect_policy #  データ収集に使用される補助的なポリシー。

In [241]:
train_env.step(np.array(2, dtype=np.int64))
# random_policy = random_tf_policy.RandomTFPolicy(train_env.time_step_spec(), train_env.action_spec()) #　オリジン
random_policy = random_tf_policy.RandomTFPolicy(train_env.time_step_spec(), train_env.action_spec())

ポリシーから行動を取得するには、`policy.action(time_step)`メソッドを呼び出します。`time_step`には、環境からの観測が含まれています。このメソッドは、3 つのコンポーネントを持つ名前付きタプルである`PolicyStep`を返します。

- `action` — 実行する行動（ここでは`0`または`1`)
- `state` — ステートフルポリシー（RNNベース）に使用
- `info` — 行動のログ確率などの補助データ

In [242]:
example_environment = tf_py_environment.TFPyEnvironment(suite_gym.wrap_env(RSP125()))
time_step = example_environment.reset()
print(time_step)
random_policy.action(time_step)

TimeStep(
{'step_type': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([0], dtype=int32)>,
 'reward': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>,
 'discount': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>,
 'observation': <tf.Tensor: shape=(1, 10), dtype=int64, numpy=array([[2, 0, 1, 2, 1, 0, 2, 1, 0, 1]])>})


PolicyStep(action=<tf.Tensor: shape=(1,), dtype=int64, numpy=array([1])>, state=(), info=())

# 指標と評価

In [245]:
#@test {"skip": true}
def compute_avg_return(environment, policy, num_episodes=10):

  total_return = 0.0
  for _ in range(num_episodes):

    time_step = environment.reset()
    episode_return = 0.0

    while not time_step.is_last():
      action_step = policy.action(time_step)
      time_step = environment.step(action_step.action)
      episode_return += time_step.reward
    total_return += episode_return

  avg_return = total_return / num_episodes
  return avg_return.numpy()[0]


# See also the metrics module for standard implementations of different metrics.
# https://github.com/tensorflow/agents/tree/master/tf_agents/metrics

In [246]:
compute_avg_return(eval_env, random_policy, num_eval_episodes)

500.0

# データ収集

In [247]:
table_name = 'uniform_table'
replay_buffer_signature = tensor_spec.from_spec(
      agent.collect_data_spec)
replay_buffer_signature = tensor_spec.add_outer_dim(
    replay_buffer_signature)

table = reverb.Table(
    table_name,
    max_size=replay_buffer_max_length,
    sampler=reverb.selectors.Uniform(),
    remover=reverb.selectors.Fifo(),
    rate_limiter=reverb.rate_limiters.MinSize(1),
    signature=replay_buffer_signature)

reverb_server = reverb.Server([table])

replay_buffer = reverb_replay_buffer.ReverbReplayBuffer(
    agent.collect_data_spec,
    table_name=table_name,
    sequence_length=2,
    local_server=reverb_server)

rb_observer = reverb_utils.ReverbAddTrajectoryObserver(
  replay_buffer.py_client,
  table_name,
  sequence_length=2)

print(agent.collect_data_spec)
print(agent.collect_data_spec._fields)

Trajectory(
{'step_type': TensorSpec(shape=(), dtype=tf.int32, name='step_type'),
 'observation': BoundedTensorSpec(shape=(10,), dtype=tf.int64, name='observation', minimum=array(0), maximum=array(2)),
 'action': BoundedTensorSpec(shape=(), dtype=tf.int64, name='action', minimum=array(0), maximum=array(2)),
 'policy_info': (),
 'next_step_type': TensorSpec(shape=(), dtype=tf.int32, name='step_type'),
 'reward': TensorSpec(shape=(), dtype=tf.float32, name='reward'),
 'discount': BoundedTensorSpec(shape=(), dtype=tf.float32, name='discount', minimum=array(0., dtype=float32), maximum=array(1., dtype=float32))})
('step_type', 'observation', 'action', 'policy_info', 'next_step_type', 'reward', 'discount')


In [248]:
print(ts.restart(train_py_env.reset()), "\n")

py_driver.PyDriver(
    env,
    py_tf_eager_policy.PyTFEagerPolicy(random_policy, use_tf_function=True),
    [rb_observer],
    max_steps=initial_collect_steps).run(
        ts.restart(train_py_env.reset())
        # time_step: tf_agents.trajectories.TimeStep,
        # policy_state: tf_agents.typing.types.NestedArray = ()
      )

TimeStep(
{'step_type': array(0, dtype=int32),
 'reward': array(0., dtype=float32),
 'discount': array(1., dtype=float32),
 'observation': array([2, 0, 1, 2, 1, 0, 2, 1, 0, 1])}) 



(TimeStep(
 {'step_type': array(1, dtype=int32),
  'reward': array(0., dtype=float32),
  'discount': array(1., dtype=float32),
  'observation': array([2, 0, 2, 0, 2, 2, 1, 0, 1, 0])}),
 ())

In [249]:
dataset = replay_buffer.as_dataset(
    num_parallel_calls=3,
    sample_batch_size=batch_size,
    num_steps=2).prefetch(3)

dataset

<_PrefetchDataset element_spec=(Trajectory(
{'step_type': TensorSpec(shape=(64, 2), dtype=tf.int32, name=None),
 'observation': TensorSpec(shape=(64, 2, 10), dtype=tf.int64, name=None),
 'action': TensorSpec(shape=(64, 2), dtype=tf.int64, name=None),
 'policy_info': (),
 'next_step_type': TensorSpec(shape=(64, 2), dtype=tf.int32, name=None),
 'reward': TensorSpec(shape=(64, 2), dtype=tf.float32, name=None),
 'discount': TensorSpec(shape=(64, 2), dtype=tf.float32, name=None)}), SampleInfo(key=TensorSpec(shape=(64, 2), dtype=tf.uint64, name=None), probability=TensorSpec(shape=(64, 2), dtype=tf.float64, name=None), table_size=TensorSpec(shape=(64, 2), dtype=tf.int64, name=None), priority=TensorSpec(shape=(64, 2), dtype=tf.float64, name=None), times_sampled=TensorSpec(shape=(64, 2), dtype=tf.int32, name=None)))>

In [250]:
iterator = iter(dataset)
print(iterator)

# eエージェント　トレーニング

In [251]:

try:
  %%time
except:
  pass

# (Optional) Optimize by wrapping some of the code in a graph using TF function.
agent.train = common.function(agent.train)

# Reset the train step.
agent.train_step_counter.assign(0)

# Evaluate the agent's policy once before training.
avg_return = compute_avg_return(eval_env, agent.policy, num_eval_episodes)
returns = [avg_return]

# Reset the environment.
# time_step = train_py_env.reset()
time_step = ts.restart(train_py_env.reset())
print(time_step)

# Create a driver to collect experience.
collect_driver = py_driver.PyDriver(
    env,
    py_tf_eager_policy.PyTFEagerPolicy(
      agent.collect_policy, use_tf_function=True),
    [rb_observer],
    max_steps=collect_steps_per_iteration)

for _ in range(num_iterations):

  # Collect a few steps and save to the replay buffer.
  time_step, _ = collect_driver.run(time_step)

  # Sample a batch of data from the buffer and update the agent's network.
  experience, unused_info = next(iterator)
  train_loss = agent.train(experience).loss

  step = agent.train_step_counter.numpy()

  if step % log_interval == 0:
    print('step = {0}: loss = {1}'.format(step, train_loss))

  if step % eval_interval == 0:
    avg_return = compute_avg_return(eval_env, agent.policy, num_eval_episodes)
    print('step = {0}: Average Return = {1}'.format(step, avg_return))
    returns.append(avg_return)

TimeStep(
{'step_type': array(0, dtype=int32),
 'reward': array(0., dtype=float32),
 'discount': array(1., dtype=float32),
 'observation': array([2, 0, 1, 2, 1, 0, 2, 1, 0, 1])})
step = 200: loss = 20799.09765625
step = 400: loss = 2125820.5
step = 600: loss = 32101610.0
step = 800: loss = 25628328.0
step = 1000: loss = 134094416.0
step = 1000: Average Return = 1000.0
step = 1200: loss = 30358918.0
step = 1400: loss = 45809224.0
step = 1600: loss = 87353896.0
step = 1800: loss = 161586784.0
step = 2000: loss = 39744456.0
step = 2000: Average Return = 1000.0
step = 2200: loss = 138104800.0
step = 2400: loss = 179948288.0
step = 2600: loss = 131347800.0
step = 2800: loss = 25003680.0
step = 3000: loss = 146783408.0
step = 3000: Average Return = 1000.0
step = 3200: loss = 123951744.0
step = 3400: loss = 17189488.0
step = 3600: loss = 54288856.0
step = 3800: loss = 78850608.0
step = 4000: loss = 71569104.0
step = 4000: Average Return = 1000.0
step = 4200: loss = 9151482.0
step = 4400: loss

KeyboardInterrupt: 

# 対戦
https://qiita.com/tanuk1647/items/7b8c2f0d09330cbfacd2#%E3%82%BD%E3%83%BC%E3%82%B9%E3%82%B3%E3%83%BC%E3%83%89

In [252]:
def visualize_hands(env):
    c = Counter(env.user_hands)
    plt.figure()
    x = np.array([c[Hand.GOO], c[Hand.CHOKI], c[Hand.PAH]])
    label = ['グー', 'チョキ', 'パー']
    plt.pie(x, labels=label, counterclock=False, startangle=90, labeldistance=0.8, autopct="%1.1f%%")
    plt.axis('equal')
    plt.title('プレーヤーの手')

    c = Counter(env.enemy_hands)
    plt.figure()
    x = np.array([c[Hand.GOO], c[Hand.CHOKI], c[Hand.PAH]])
    label = ['グー', 'チョキ', 'パー']
    plt.pie(x, labels=label, counterclock=False, startangle=90, labeldistance=0.8, autopct="%1.1f%%")
    plt.axis('equal')
    plt.title('敵の手')


def visualize_positions(env):
    plt.figure()
    x = range(1, len(env.user_positions) + 1)
    u = env.user_positions
    e = env.enemy_positions
    plt.plot(x, u, marker='.', label='プレーヤーの位置')
    plt.plot(x, e, marker='.', label='敵の位置')
    plt.legend(loc='best', fontsize=10)
    plt.grid()
    plt.xlabel('steps')
    plt.ylabel('position')
    plt.show()

# 皺寄せ

In [ ]:
class TrainIntervalLogger2():
    def __init__(self, interval=10000):
        super().__init__(interval=interval)
        self.records = {}

    def on_train_begin(self, logs):
        super().on_train_begin(logs)
        self.records['interval'] = []
        self.records['episode_reward'] = []
        for metrics_name in self.metrics_names:
            self.records[metrics_name] = []

    def on_step_begin(self, step, logs):
        if self.step % self.interval == 0:
            if len(self.episode_rewards) > 0:
                self.records['interval'].append(self.step // self.interval)
                self.records['episode_reward'].append(np.mean(self.episode_rewards))
                metrics = np.array(self.metrics)
                assert metrics.shape == (self.interval, len(self.metrics_names))
                if not np.isnan(metrics).all():  # not all values are means
                    means = np.nanmean(self.metrics, axis=0)
                    assert means.shape == (len(self.metrics_names),)
                    for name, mean in zip(self.metrics_names, means):
                        self.records[name].append(mean)
        super().on_step_begin(step, logs)

class DQNRSP125:
    # 重み保存先
    weightdir = './data'
    weightfile = './data/dqn_{}_{}_weights.h5'

    # モデルの初期化
    def __init__(self, strategy=Strategy.HUMAN, recycle=True):
        print('モデルを作成します。')
        self.train_interval_logger = None

        # Get the environment and extract the number of actions.
        self.env = RSP125(strategy=strategy)
        self.env_name = 'janken_125'
        self.weightfile = DQNRSP125.weightfile.format(self.env_name, str(strategy))
        self.nb_actions = self.env.action_space.n

        # Next, we build a very simple model.
        self.model = Sequential()
        self.model.add(Flatten(input_shape=(1,) + self.env.observation_space.shape))
        self.model.add(Dense(128))
        self.model.add(Activation('relu'))
        self.model.add(Dense(self.nb_actions))
        self.model.add(Activation('linear'))
        #print(self.model.summary())

        # Finally, we configure and compile our agent.
        # You can use every built-in Keras optimizer and even the metrics!
        memory = SequentialMemory(limit=50000, window_length=1)
        policy = BoltzmannQPolicy(tau=1.)
        self.dqn = DQNAgent(model=self.model, nb_actions=self.nb_actions, memory=memory,
                            nb_steps_warmup=1000, target_model_update=1e-2, policy=policy)
        self.dqn.compile(Adam(lr=1e-3), metrics=[])

        self.__istrained = False
        print('モデルを作成しました。')

        if recycle:
            if exists(self.weightfile):
                try:
                    print('訓練済み重みを読み込みます。')
                    self.dqn.load_weights(self.weightfile)
                    self.__istrained = True
                    print('訓練済み重みを読み込みました。')
                    return None
                except:
                    print('訓練済み重みの読み込み中にエラーが発生しました。')
                    print('Unexpected error:', exc_info()[0])
                    raise
            else:
                print('訓練済み重みが存在しません。訓練を行ってください。')

    # 訓練
    def train(self, nb_steps=30000, verbose=1, visualize=False, log_interval=3000):
        if self.__istrained:
            raise RuntimeError('このモデルは既に訓練済みです。')

        print('訓練を行うので、お待ちください。')

        # 訓練実施
        # Okay, now it's time to learn something!
        # We visualize the training here for show, but this slows down training quite a lot.
        # You can always safely abort the training prematurely using Ctrl + C.
        callbacks = []
        if verbose == 1:
            self.train_interval_logger = TrainIntervalLogger2()
            callbacks.append(self.train_interval_logger)
            verbose = 0
        elif verbose > 1:
            callbacks.append(TrainEpisodeLogger())
            verbose = 0

        hist = self.dqn.fit(self.env, nb_steps=nb_steps,
                            callbacks=callbacks, verbose=verbose,
                            visualize=visualize, log_interval=log_interval)
        self.__istrained = True

        if self.train_interval_logger is not None:
            # 訓練状況の可視化
            interval = self.train_interval_logger.records['interval']
            episode_reward = self.train_interval_logger.records['episode_reward']
            mean_q = self.train_interval_logger.records['mean_q']
            if len(interval) > len(mean_q):
                mean_q = np.pad(mean_q, [len(interval) - len(mean_q), 0], "constant")
            plt.figure()
            plt.plot(interval, episode_reward, marker='.', label='報酬')
            plt.plot(interval, mean_q, marker='.', label='Q値')
            plt.legend(loc='best', fontsize=10)
            plt.grid()
            plt.xlabel('interval')
            plt.ylabel('score')
            plt.xticks(np.arange(min(interval),
                                 max(interval) + 1,
                                 (max(interval) - min(interval))//7))
            plt.show()

        # 重みの保存
        if not exists(DQNJankenGlico.weightdir):
            try:
                mkdir(DQNJankenGlico.weightdir)
            except:
                print('重み保存フォルダの作成中にエラーが発生しました。')
                print('Unexpected error:', exc_info()[0])
                raise
        try:
            # After training is done, we save the final weights.
            self.dqn.save_weights(self.weightfile, overwrite=True)
        except:
            print('重みの保存中にエラーが発生しました。')
            print('Unexpected error:', exc_info()[0])
            raise

        return hist

    # テスト
    def test(self, nb_episodes=10, visualize=True, verbose=1):
        # Finally, evaluate our algorithm for 5 episodes.
        hist = self.dqn.test(self.env, nb_episodes=nb_episodes,
                             verbose=verbose, visualize=visualize)
        return hist

def visualize_hands(env):
    c = Counter(env.user_hands)
    plt.figure()
    x = np.array([c[Hand.R], c[Hand.S], c[Hand.P]])
    label = ['グー', 'チョキ', 'パー']
    plt.pie(x, labels=label, counterclock=False, startangle=90, labeldistance=0.8, autopct="%1.1f%%")
    plt.axis('equal')
    plt.title('プレーヤーの手')

    c = Counter(env.enemy_hands)
    plt.figure()
    x = np.array([c[Hand.R], c[Hand.S], c[Hand.P]])
    label = ['グー', 'チョキ', 'パー']
    plt.pie(x, labels=label, counterclock=False, startangle=90, labeldistance=0.8, autopct="%1.1f%%")
    plt.axis('equal')
    plt.title('敵の手')


def visualize_positions(env):
    plt.figure()
    x = range(1, len(env.user_positions) + 1)
    u = env.user_positions
    e = env.enemy_positions
    plt.plot(x, u, marker='.', label='プレーヤーの位置')
    plt.plot(x, e, marker='.', label='敵の位置')
    plt.legend(loc='best', fontsize=10)
    plt.grid()
    plt.xlabel('steps')
    plt.ylabel('position')
    plt.show()

In [ ]:
d = DQNRSP125(strategy=Strategy.RANDOM, recycle=False)
h = d.train(nb_steps=40000, log_interval=2000, verbose=1)


モデルを作成します。


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


NameError: name 'SequentialMemory' is not defined

# メモ

作りたい環境<br>
入力：
- 5回前の自分の手(0,1,2)
- 5回前の相手の手(0,1,2)
-  /
-  /
-  /
- 1回前の自分の手(0,1,2)
- 1回前の相手の手(0,1,2)
3^10
出力：
- 自分の出す手(0,1,2)